In [ ]:
# 初めの一回だけこのセルを実行してください、データセットをダウンロードして展開します
# 一回実行すれば、データセットはダウンロードされたままなので、再起動後等再び実行する必要はありません
import urllib.request
import zipfile

# URLを指定
url = "https://storage.googleapis.com/tutor-contents-dataset/5050_nlp_data.zip"
save_name = url.split('/')[-1]

# ダウンロードする
mem = urllib.request.urlopen(url).read()

# ファイルへ保存
with open(save_name, mode='wb') as f:
    f.write(mem)

# zipファイルをカレントディレクトリに展開する
zfile = zipfile.ZipFile(save_name)
zfile.extractall('.')

# 総合添削問題

#### 問題

- 学習データに出現する単語をテキストの特徴量とし、ランダムフォレスト、ナイーブベイズの2つの分類器を学習させ、livedoor newsコーパスでの精度を評価してください。
- また、テキストの特徴量を名詞、動詞、形容詞、形容動詞のみに限定した時のモデルの精度を同様に評価してください。

#### ヒント

- ナイーブベイズ分析には、`sklearn.naive_bayes.MultinomialNB()`を使用してください
<br>[参考]　scikit-learn公式ドキュメント https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [ ]:
import glob
import time
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from janome.tokenizer import Tokenizer


def load_livedoor_news_corpus():
    category = {
        'dokujo-tsushin': 1,
        'it-life-hack': 2,
        'kaden-channel': 3,
        'livedoor-homme': 4,
        'movie-enter': 5,
        'peachy': 6,
        'smax': 7,
        'sports-watch': 8,
        'topic-news': 9
    }
    docs = []
    labels = []

    for c_name, c_id in category.items():
        files = glob.glob("./5050_nlp_data/{c_name}/{c_name}*.txt".format(c_name=c_name))

        text = ''
        for file in files:
            with open(file, 'r', errors='ignore') as f:
                lines = f.read().splitlines()

                # 1,2行目に書いたあるURLと時間は関係ないので取り除きます。
                url = lines[0]
                datetime = lines[1]
                subject = lines[2]
                body = "".join(lines[3:])
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels


docs, labels = load_livedoor_news_corpus()

# indices は0からドキュメントの数までの整数をランダムに並べ替えた配列
random.seed()
indices = list(range(len(docs)))
# 9割をトレーニングデータとする
separate_num = int(len(docs) * 0.9)

random.shuffle(indices)

train_data = [docs[i] for i in indices[0:separate_num]]
train_labels = [labels[i] for i in indices[0:separate_num]]
test_data = [docs[i] for i in indices[separate_num:]]
test_labels = [labels[i] for i in indices[separate_num:]]

# Tf-idfを用いてtrain_dataをベクトル化してください
vectorizer = TfidfVectorizer()
train_matrix = vectorizer.fit_transform(train_data)

# ナイーブベイズを用いて分類をおこなってください。
clf = MultinomialNB()
clf.fit(train_matrix, train_labels)

# ランダムフォレストを用いて分類をおこなってください
clf2 = RandomForestClassifier(n_estimators=2)
clf2.fit(train_matrix, train_labels)

# テストデータを変換
test_matrix = vectorizer.transform(test_data)

# 分類結果を表示
print(clf.score(train_matrix, train_labels))
print(clf.score(test_matrix, test_labels))
print(clf2.score(train_matrix, train_labels))
print(clf2.score(test_matrix, test_labels))


# 単語の抽出
def tokenize(text):
    t = Tokenizer()
    tokens = t.tokenize(','.join(text))
    noun = []
    for token in tokens:
        # 「名詞」「動詞」「形容詞」「形容動詞」を取り出してください
        part_of_speech = token.part_of_speech.split(",")[0]
        if part_of_speech == "名詞" or part_of_speech == "動詞" or part_of_speech == "形容詞" or part_of_speech == "形容動詞":
            noun.append(token.surface)
    return noun


# 単語の抽出して学習
vectorizer = TfidfVectorizer(tokenizer=tokenize(docs))
train_matrix = vectorizer.fit_transform(train_data)
test_matrix = vectorizer.transform(test_data)
clf.fit(train_matrix, train_labels)
clf2.fit(train_matrix, train_labels)

# 結果を表示
if __name__ == '__main__':
    start = time.time()
    print(clf.score(train_matrix, train_labels))
    print(clf.score(test_matrix, test_labels))
    print(clf2.score(train_matrix, train_labels))
    print(clf2.score(test_matrix, test_labels))
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time) + "[sec]")


##  解答例

In [ ]:
import glob
import time
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from janome.tokenizer import Tokenizer


def load_livedoor_news_corpus():
    category = {
        'dokujo-tsushin': 1,
        'it-life-hack': 2,
        'kaden-channel': 3,
        'livedoor-homme': 4,
        'movie-enter': 5,
        'peachy': 6,
        'smax': 7,
        'sports-watch': 8,
        'topic-news': 9
    }
    docs = []
    labels = []

    for c_name, c_id in category.items():
        files = glob.glob("./5050_nlp_data/{c_name}/{c_name}*.txt".format(c_name=c_name))

        text = ''
        for file in files:
            with open(file, 'r', errors='ignore') as f:
                lines = f.read().splitlines()

                # 1,2行目に書いたあるURLと時間は関係ないので取り除きます。
                url = lines[0]
                datetime = lines[1]
                subject = lines[2]
                body = "".join(lines[3:])
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels


docs, labels = load_livedoor_news_corpus()

# indices は0からドキュメントの数までの整数をランダムに並べ替えた配列
random.seed()
indices = list(range(len(docs)))
# 9割をトレーニングデータとする
separate_num = int(len(docs) * 0.9)

random.shuffle(indices)

train_data = [docs[i] for i in indices[0:separate_num]]
train_labels = [labels[i] for i in indices[0:separate_num]]
test_data = [docs[i] for i in indices[separate_num:]]
test_labels = [labels[i] for i in indices[separate_num:]]

# Tf-idfを用いてtrain_dataをベクトル化してください
vectorizer = TfidfVectorizer()
train_matrix = vectorizer.fit_transform(train_data)

# ナイーブベイズを用いて分類をおこなってください。
clf = MultinomialNB()
clf.fit(train_matrix, train_labels)

# ランダムフォレストを用いて分類をおこなってください
clf2 = RandomForestClassifier(n_estimators=100)
clf2.fit(train_matrix, train_labels)

# テストデータを変換
test_matrix = vectorizer.transform(test_data)

# 分類結果を表示
print(clf.score(train_matrix, train_labels))
print(clf.score(test_matrix, test_labels))
print(clf2.score(train_matrix, train_labels))
print(clf2.score(test_matrix, test_labels))


# 単語の抽出
def tokenize(text):
    t = Tokenizer()
    tokens = t.tokenize(','.join(text))
    noun = []
    for token in tokens:
        # 「名詞」「動詞」「形容詞」「形容動詞」を取り出してください
        partOfSpeech = token.part_of_speech.split(',')[0]

        if partOfSpeech == '名詞':
            noun.append(token.surface)
        if partOfSpeech == '動詞':
            noun.append(token.surface)
        if partOfSpeech == '形容詞':
            noun.append(token.surface)
        if partOfSpeech == '形容動詞':
            noun.append(token.surface)
    return noun


# 単語の抽出して学習
vectorizer = TfidfVectorizer(tokenizer=tokenize(docs))
train_matrix = vectorizer.fit_transform(train_data)
test_matrix = vectorizer.transform(test_data)
clf.fit(train_matrix, train_labels)
clf2.fit(train_matrix, train_labels)

# 結果を表示
if __name__ == '__main__':
    start = time.time()
    print(clf.score(train_matrix, train_labels))
    print(clf.score(test_matrix, test_labels))
    print(clf2.score(train_matrix, train_labels))
    print(clf2.score(test_matrix, test_labels))
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time) + "[sec]")

***